总结：

| 特性                     | FunctionCallingAgent                      | ReActAgent                            | StructuredPlannerAgent             |
|--------------------------|-------------------------------------------|---------------------------------------|------------------------------------|
| **设计目的**             | 直接调用函数来完成任务。无需复杂的规划和推理过程。| 使用结构化思考（ReAct循环）来分解复杂问题并逐步解决它。     | 先进行整体规划，然后再执行具体步骤。 |
| **LLM交互方式**          | 通过工具直接与LLM交互，不解析中间的思维步骤。| 向LLM提供明确的提示格式和结构化输出格式（如Thought、Action等）。| 先创建计划，然后逐步执行任务，调整计划。|
| **使用prompt的方式**     | 默认无自定义prompt，由工具驱动。         | 明确定义了思考和行动规则的prompt。   | 创建和优化整体任务规划的特定提示。  |
| **思维过程**             | 简单直接，每个步骤即为一个明确的任务执行。| 结构化、分步地处理问题（分解、推理）。      | 先全局计划，后逐步执行并调整。       |
| **灵活性和复杂性管理**   | 较低的灵活性和复杂的任务解决能力。         | 高度灵活且适合解决复杂任务。           | 适用于需要先期规划的任务解决方式。    |
| **适用场景**             | 简单直接的问题解决或函数执行。            | 处理涉及多步骤、推理和问题分解的复杂查询。       | 先进行详细规划，然后逐步实施计划。   |

这个表格总结了三种不同类型的Agent在设计目的上的主要区别，以及它们与LLM交互的方式、使用的提示方式、思维过程、灵活性和复杂性管理能力方面的差异。

- **FunctionCallingAgent** 适用于简单的任务执行，其优点在于直接和高效。
- **ReActAgent** 则更适合于需要结构化思考和多步骤推理的场景，能够有效地处理复杂的查询或问题分解任务。
- **StructuredPlannerAgent** 更适合那些在开始时就需要详细规划的任务，并且可以动态调整计划以适应变化的需求。

每种类型的Agent都有其适用的特定情境，选择合适的类型有助于更高效地解决问题。

---

在使用llamaIndex时，有3种不同类型的Agent，这3种Agent的原理及区别




In [1]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.core import agent
indexs=list(filter(lambda att:att.endswith('Agent')>0,dir(agent)))
print(indexs)

['FunctionCallingAgent', 'ReActAgent', 'StructuredPlannerAgent']


通过查看源代码，他们之间的关系如下：

```mermaid
classDiagram
	class BaseAgent
	class BaseAgentRunner
	BaseAgent <|-- BaseAgentRunner
	
	class AgentRunner
	class BaseAgentWorker
	class ReActAgent
	class ReActAgentWorker
	
	BaseAgentRunner<|-- AgentRunner
	AgentRunner<|-- ReActAgent
	ReActAgent<-- ReActAgentWorker
	BaseAgentWorker <|-- ReActAgentWorker
	
	class StructuredPlannerAgent
	class BasePlanningAgentRunner
	BasePlanningAgentRunner <|-- StructuredPlannerAgent
	StructuredPlannerAgent<-- BaseAgentWorker
	AgentRunner<|-- BasePlanningAgentRunner
	
	class FunctionCallingAgent
	AgentRunner <|-- FunctionCallingAgent
	class FunctionCallingAgentWorker
	FunctionCallingAgent <-- FunctionCallingAgentWorker
	BaseAgentWorker <| -- FunctionCallingAgentWorker
	
```

这里有3大类的类`Worker、Runner、Agent`，其中Agent继承Runner，Agent使用Worker作为执行步骤



In [2]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

base_url='http://localhost:11434'
llm = Ollama(model="qwen2.5:latest", request_timeout=360.0,base_url=base_url)
Settings.llm = llm
Settings.embed_model = OllamaEmbedding(model_name="quentinz/bge-large-zh-v1.5:latest",base_url=base_url)

In [3]:
from llama_index.core.tools import FunctionTool

def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b
multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b
add_tool = FunctionTool.from_defaults(fn=add)

## FunctionCallingAgent与ReActAgent

In [4]:
from llama_index.core.agent import FunctionCallingAgent

function_calling_agent=FunctionCallingAgent.from_tools(tools=[multiply_tool, add_tool],verbose=True)
response = function_calling_agent.chat("计算结果，1000+157*2？")
print(response)

> Running step f1466f93-b140-4f58-b800-b7221e1fe5cd. Step input: 计算结果，1000+157*2？
Added user message to memory: 计算结果，1000+157*2？
=== Calling Function ===
Calling function: multiply with args: {"a": 157, "b": 2}
=== Function Output ===
314
> Running step 4a7cdcf8-97b5-4e1b-9387-62734c68de34. Step input: None
=== Calling Function ===
Calling function: add with args: {"a": 1000, "b": 314}
=== Function Output ===
1314
> Running step 600c57bf-c783-49fc-be90-f402cd35e78a. Step input: None
=== LLM Response ===
计算结果是 \( 1000 + 157 \times 2 = 1314 \)。
计算结果是 \( 1000 + 157 \times 2 = 1314 \)。


In [5]:
from llama_index.core.agent import ReActAgent
# 创建代理 
reAct_agent = ReActAgent.from_tools([multiply_tool, add_tool], verbose=True)
response = reAct_agent.chat("计算结果，1000+157*2？")
print(response)

> Running step 13973598-2e38-4cc3-ba70-0897d09bb55c. Step input: 计算结果，1000+157*2？
Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 157, 'b': 2}
Observation: 314
> Running step eb577013-9af9-4864-8516-ca97b24d288f. Step input: None
Thought: I can now perform the addition using the result from the multiplication.
Action: add
Action Input: {'a': 1000, 'b': 314}
Observation: 1314
> Running step e75c52b9-109a-4fdc-b555-6159e9c3fb5c. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 计算结果是1314。
计算结果是1314。


```python
task = self.create_task(message)

result_output = None
dispatcher.event(AgentChatWithStepStartEvent(user_msg=message))
while True:
    # pass step queue in as argument, assume step executor is stateless
    cur_step_output = self._run_step(
        task.task_id, mode=mode, tool_choice=tool_choice
    )

    if cur_step_output.is_last:
        result_output = cur_step_output
        break

    # ensure tool_choice does not cause endless loops
    tool_choice = "auto"
```

1. FunctionCallingAgent 与 ReActAgent 的chat均是使用以上代码，看得出来是一个一直输出的过程，除非输出出现结束标志(cur_step_output.is_last=True)
2. _run_step步骤开始出现不同：
    - 两者都是分为3个步骤：(1)执行一次llm推理；(2)解析llm输出；(3) 生成下一步的Task
    - 在执行llm推理时，FunctionCallingAgent直接调用接口chat_with_tools，而ReActAgent是chat，也就是FunctionCallingAgent提供tool给llm，而ReActAgent只是通过prompt提供
    - 解析llm输出时，通过查看ReActAgent的prompt，可以看出其要求结构化输出，解析是提取输出“Thought、Action、Action Input、Observation”的不同输出
    - 根据解析的输出，规划下一步输出

![alt text](../../doc/functionCallingVSReActAgent.png)


In [6]:
print(function_calling_agent.get_prompts())
print(reAct_agent.get_prompts())

{}
{'agent_worker:system_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['tool_desc', 'tool_names', '"input": "hello world", "num_beams": 5', "'input': 'hello world', 'num_beams': 5"], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n{tool_desc}\n\n\n## Output Format\n\nPlease answer in the same language as the question and use the following format:\n\n```\nThought: The current language of the user is: (user\'s language). I need to use a tool to help me answer the q

FunctionCallingAgent没有自定义prompt，而是直接提供tool供llm内部选择，ReActAgent规范了llm的思考规则，通过选择工具回答问题，并规则输出格式，让我仔细看看prompt的内容

In [7]:
system_prompt=reAct_agent.get_prompts()['agent_worker:system_prompt']
print(system_prompt.template)

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may use code markers within your response if y

ReActAgent的prompt包含2部分：

Tools：明确能使用的工具

Output Format：明确输出的格式，包含2个分支

1. 如果是中间步骤，按照以下格式输出：
```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```
2. 如果是最终结果，按照以下格式之一输出
```
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: [your answer here (In the same language as the user's question)]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: [your answer here (In the same language as the user's question)]
```

可见ReActAgent是一个规范思维方式的，自定义程度比FunctionCallingAgent高的Agent

## StructuredPlannerAgent

In [12]:
from llama_index.core.agent import StructuredPlannerAgent,FunctionCallingAgentWorker

# 创建代理 
# create the function calling worker for reasoning
worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, add_tool], verbose=True
)

# wrap the worker in the top-level planner
agent = StructuredPlannerAgent(
    worker, tools=[multiply_tool, add_tool],
    verbose=True,
    memory=None
)

response = agent.chat("计算结果，1000+157*2？")

=== Initial plan ===
计算乘法部分:
157 * 2 -> 314.0
deps: []


计算加法部分:
1000 + 314.0 -> 1314.0
deps: ['计算乘法部分']


> Running step 3cd2cc1c-0f45-4fb1-aad7-d28b4c821f2c. Step input: 157 * 2
Added user message to memory: 157 * 2
=== Calling Function ===
Calling function: multiply with args: {"a": 157, "b": 2}
=== Function Output ===
314
> Running step 95f3f0a6-8ba7-4dbc-9e86-f33118f69ad5. Step input: None
=== LLM Response ===
The product of 157 and 2 is 314.
=== Refined plan ===
计算加法部分:
1000 + 314.0 -> 1314.0
deps: ['计算乘法部分']


完成任务:
 -> 1314.0
deps: ['计算加法部分']


> Running step abad3f05-a750-4f62-85bd-28db75fc4fb6. Step input: 1000 + 314.0
Added user message to memory: 1000 + 314.0
=== Calling Function ===
Calling function: add with args: {"a": 1000, "b": 314}
=== Function Output ===
1314
> Running step 203c38a5-29fd-4bce-acac-5c3530a25ed3. Step input: None
=== LLM Response ===
The sum of 1000 and 314.0 is 1314.0.
=== Refined plan ===
验证最终结果:
 -> 1314.0
deps: ['计算加法部分']


> Running step bee84f9a-

和前面两个“边规划边执行不同”，还有一种思维方式，提前规划好，然后执行规划，得到最终答案， StructuredPlannerAgent 就是这种方式，该Agent整体运行逻辑如下

![alt text](../../doc/PlanningAgentRunner.png)

1. create_plan：提出整体计划
2. 执行：执行第一个任务
3. 优化计划：根据上一任务结果及下一任务，调整优化计划

仔细地，还是从prompt了解其原理

```python
DEFAULT_INITIAL_PLAN_PROMPT = """\
Think step-by-step. Given a task and a set of tools, create a comprehesive, end-to-end plan to accomplish the task.
Keep in mind not every task needs to be decomposed into multiple sub-tasks if it is simple enough.
The plan should end with a sub-task that satisfies the overall task.

The tools available are:
{tools_str}

Overall Task: {task}
"""
```

这是StructuredPlannerAgent的`create_plan`方法的prompt，可以看出其作用是提出整体规划

还有一个优化计划的prompt，作用是根据已执行的任务结果，更新后续任务

```python
DEFAULT_PLAN_REFINE_PROMPT = """\
Think step-by-step. Given an overall task, a set of tools, and completed sub-tasks, update (if needed) the remaining sub-tasks so that the overall task can still be completed.
The plan should end with a sub-task that satisfies the overall task.
If the remaining sub-tasks are sufficient, you can skip this step.

The tools available are:
{tools_str}

Overall Task:
{task}

Completed Sub-Tasks + Outputs:
{completed_outputs}

Remaining Sub-Tasks:
{remaining_sub_tasks}
"""
```

### 创建初始任务和计划

以下根据StructuredPlannerAgent原理，使用低级API展示

In [9]:
plan_id = agent.create_plan("计算结果，1000+157*2？")

print('------------'*3)
plan = agent.state.plan_dict[plan_id]
for sub_task in plan.sub_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

=== Initial plan ===
Step 1: Multiply 157 by 2:
multiply(157, 2) -> PENDING
deps: []


Step 2: Add the result of Step 1 to 1000:
add(1000, {result from Step 1}) -> PENDING
deps: ['Step 1']


------------------------------------
===== Sub Task Step 1: Multiply 157 by 2 =====
Expected output:  PENDING
Dependencies:  []
===== Sub Task Step 2: Add the result of Step 1 to 1000 =====
Expected output:  PENDING
Dependencies:  ['Step 1']


### 执行第一组任务

In [10]:
# 获取下一步要执行的任务
next_tasks = agent.state.get_next_sub_tasks(plan_id)
for sub_task in next_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

# 执行任务
for sub_task in next_tasks:
    response = agent.run_task(sub_task.name)
    agent.mark_task_complete(plan_id, sub_task.name)

===== Sub Task Step 1: Multiply 157 by 2 =====
Expected output:  PENDING
Dependencies:  []
> Running step 6008e40b-5e57-471f-9a0d-94c422c5c9be. Step input: multiply(157, 2)
Added user message to memory: multiply(157, 2)
=== Calling Function ===
Calling function: multiply with args: {"a": 157, "b": 2}
=== Function Output ===
314
> Running step dcf22d0b-c100-4a88-83ce-a83bbd82f485. Step input: None
=== LLM Response ===
The product of 157 and 2 is 314.


### 查看是否结束

In [11]:
next_tasks = agent.get_next_tasks(plan_id)
print(len(next_tasks))

0


### 优化任务

In [12]:
# refine the plan
agent.refine_plan(
    "计算结果，1000+157*2？",
    plan_id,
)

=== Refined plan ===
Step 2: Add the result of Step 1 to 1000:
add(1000, 314) -> PENDING
deps: ['Step 1']


